In [ ]:
import pandas as pd
from pathlib import Path
from time import sleep

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.optimize import curve_fit
from scipy.stats import percentileofscore
from scipy.interpolate import CubicSpline, PchipInterpolator, Akima1DInterpolator, \
                              make_interp_spline, CubicHermiteSpline, BSpline, splrep

import statsmodels.api as sm

# Contents
 - [Market Data](#Market-Data)
 - [Market Draft Joins](#Market-Draft-Joins)
   - [Position Label Cleaning](#Position-Label-Cleaning)
 - [Tables and Graphs](#Tables-and-Graphs)
   - [Final Table](#Final-Table)
   - [Market Fit Graphs](#Market-Fit)
   - [Market Explore Graphs](#Market-Explore)

## Market Data
 - contract history for position from OTC

In [ ]:
market_data = pd.DataFrame()

positions = {
    'QB':'quarterback', 'RB':'running-back', 'WR':'wide-receiver', 'TE':'tight-end',
    'T': ['left-tackle','right-tackle'], 'G': ['left-guard', 'right-guard'],
    'C': 'center', 'iDL':'interior-defensive-line', 'EDGE':'edge-rusher',
    'LB': 'linebacker', 'CB':'cornerback', 'S':'safety'
            }

for pos, pos_name in tqdm(positions.items()):
    if type(pos_name) == str:
        pos_name = [pos_name]
    for name in pos_name:
        d = pd.read_html(f'https://overthecap.com/contract-history/{name}')[0]
        # drop blank and unused
        drop_col = d.columns[d.columns.str.startswith('Unnamed:')]
        drop_col = drop_col.append(d.columns[d.columns.str.startswith('Inflated')])    
        d.drop(columns=drop_col, inplace=True)
        d.drop(columns=['Team','Years'], inplace=True)

        # rename "APY as % of Cap..." and "Year Signed"
        d.rename(columns={d.columns[-1]:'Perc_Cap',
                 'Year Signed':'Year'}, inplace=True)
        d.Perc_Cap = d.Perc_Cap.str.replace('%','').astype(float)

        # convert $ to int
        for col in ['Value','APY','Guaranteed']:
            d[col] = d[col].str.replace('$','').str.replace(',','').astype(int)

        # drop bogus years and 0% cap, add position
        d = d[d['Year']>0]
        d = d[d['Perc_Cap']>0]
        d.loc[:,'Position'] = pos

        market_data = pd.concat([market_data, d], ignore_index=True)
    sleep(0.5)
    
market_data.to_parquet(Path('market based position groups','data', 'OTC_contract_history.parquet'))

## Market Draft Joins
 - match draft player to veteran market deals based on name, position

In [ ]:
# possible position matches
pos_match = {
    # market_data to acceptable draft positions
    'CB': ['CB','DB','S'],
    'S': ['CB','DB','S'],
    'LB': ['DE','ILB','OLB','LB'],
    'iDL': ['DE','DL','DT','NT'],
    'EDGE': ['ILB','OLB','DE','DL','LB'],
    'C': ['C','G','OL'],
    'G': ['C','G','OL','T'],
    'T': ['T','G','OL'],
    'RB': ['RB','FB',],
    'WR': ['WR','TE'],    
    'TE': ['WR','TE','FB'],
    'QB':['QB'],
}

In [2]:
folder = Path('data')


# cap table, pick vs salary cap percentage
cap = pd.read_csv(Path(folder, 'pick_cap_percentage.csv'), index_col=0)
# salary cap vs year
cap_history = pd.read_csv(Path(folder, 'salary_cap_history.csv'), index_col=0)
cap_history = cap_history[cap_history.Year>=2014]

# draft table
draft = pd.read_csv(Path(folder, 'draft_2002-2023.csv'), index_col=0)

# draft adjustments, 2011 and after for CB agreement --> wagescale
draft = draft[draft.draft_year >= 2011]
draft.drop(columns = draft.columns[13:27], inplace=True)
draft.drop(columns=draft.columns[draft.columns.str.startswith('Misc_')], inplace=True)
draft.rename(columns={'Approx Val_wAV':'wAV', 'Approx Val_DrAV':'dAV', 'G':'Games','Uni':'College'}, inplace=True)

# clean negatives | draft.loc[[2744,3295,4251],:]
draft.loc[draft[draft.dAV<0].index, 'dAV'] = 0
draft.loc[draft[draft.wAV<0].index, 'wAV'] = 0

# normalize value by games played
draft.loc[:,'wAVpG'] = draft['wAV']/draft['Games']
draft.loc[:,'dAVpG'] = draft['dAV']/draft['Games']
# fill na with 0
draft['wAVpG'] = draft['wAVpG'].fillna(0)
draft['dAVpG'] = draft['dAVpG'].fillna(0)

# Change "To" to years played
draft.loc[:,'To'] = draft['To']-draft['draft_year']+1
draft.rename(columns={'To':'Yrs', 'draft_year':'Drafted'}, inplace=True)
draft.loc[draft[draft.Yrs<=0].index, 'Yrs'] = 0

# Convert single Pos==OT (AJ Arcuri) to T
draft.loc[draft[draft.Pos=='OT'].index,'Pos'] = 'T'

# Add Cap Percentage
draft.loc[:,'Cap'] = draft['Pick'].map(lambda x: cap.loc[x, 'Perc'])

In [ ]:
tester = draft.copy()
for ind in tqdm(tester.index):
    draft_name, draft_pos, draft_year = tester.loc[ind,'Player'], tester.loc[ind,'Pos'], tester.loc[ind,'Drafted']
    contracts = market_data[(market_data.Player == draft_name) & (market_data.Year != draft_year)]
    contracts = contracts[contracts.Year != draft_year]

    if contracts.Position.unique().shape[0] > 1:
        drop_pos = []
        for pos_check in contracts.Position.unique():
            if draft_pos not in pos_match[pos_check]:
                drop_pos.append(pos_check)
        contracts = contracts[~contracts.Position.isin(drop_pos)]
        
    if contracts.empty:
        tester.loc[ind,'vet_contracts'] = 0
        tester.loc[ind,'market_Pos'] = 'N/A'
        tester.loc[ind,'next_Cap'] = 0      
        tester.loc[ind,'max_Cap'] = 0 
    tester.loc[ind,'market_Pos'] = ','.join(contracts.Position.unique())
    tester.loc[ind,'vet_contracts'] = contracts.Year.unique().shape[0]
    tester.loc[ind,'next_Cap'] = contracts[contracts.Year==contracts.Year.min()].Perc_Cap.max()
    tester.loc[ind,'max_Cap'] = contracts.Perc_Cap.max()

tester['vet_contracts'] = tester['vet_contracts'].astype(int)        

In [ ]:
tester.dropna(inplace=True, ignore_index=True)
tester.to_parquet(Path('market based position groups', 'data', 'draft_market_join.parquet'))

In [ ]:
# copy back if satisfied with results
draft = tester.copy()

### Position Label Cleaning
 - choose one position from matched contracts

In [ ]:
market_data = pd.read_parquet(Path('market based position groups','data', 'OTC_contract_history.parquet'))

In [ ]:
for ind in tester[tester.market_Pos.str.find(',')>-1].index:
    draft_name, draft_pos, draft_year = tester.loc[ind,'Player'], tester.loc[ind,'Pos'], tester.loc[ind,'Drafted']
    contracts = market_data[(market_data.Player == draft_name) & (market_data.Year != draft_year)]
    contracts = contracts[contracts.Year != draft_year]

    if contracts.Position.unique().shape[0] > 1:
        drop_pos = []
        for pos_check in contracts.Position.unique():
            if draft_pos not in pos_match[pos_check]:
                drop_pos.append(pos_check)
        contracts = contracts[~contracts.Position.isin(drop_pos)]    
        
    if contracts.Position.value_counts().values[0] > contracts.Position.value_counts().values[1]:
        tester.loc[ind,'market_Pos'] = contracts.Position.value_counts().index[0]
    else:
        tester.loc[ind,'market_Pos'] = contracts.sort_values('Year', ascending=False).Position.values[0]

In [ ]:
tester.to_parquet(Path('market based position groups', 'data', 'draft_market_join_1pos.parquet'))

## Tables and Graphs
 - robust linear model fit for veteran market
 - second contract, max contract vs performance **wAVpG**
 - second contract - rookie contract vs performance

In [ ]:
data = pd.read_parquet(Path('market based position groups', 'data', 'draft_market_join_1pos.parquet'))
data.loc[:,'delta'] = data.next_Cap-data.Cap # second contract - rookie deal

In [ ]:
def pos_cutoffs(pos, cap_type):
    if cap_type == 'max':
        if pos in ['WR','RB']:
            cutoff_percentile = 85 
        elif pos in ['CB','iDL','LB','EDGE','S','QB']:
            cutoff_percentile = 80
        elif pos in ['TE',]:
            cutoff_percentile = 75
        elif pos in ['T','C']:
            cutoff_percentile = 65        
        else: # G
            cutoff_percentile = 70   
    elif cap_type == 'next':
        if pos in ['RB','S','iDL','CB']:
            cutoff_percentile = 88 
        elif pos =='EDGE':
            cutoff_percentile = 85
        elif pos == 'C':
            cutoff_percentile = 70
        else:
            cutoff_percentile = 80        
    else:
        cutoff_percentile = None
    return cutoff_percentile

### Final Table
 - fit parameters, bounds, value and overpay picks
 - once for second contract `py='next_Cap'`, one for max contract `py='max_Cap'`

In [ ]:
fit_summary = pd.DataFrame()

for pos in ['WR', 'CB', 'iDL', 'LB', 'EDGE', 'S', 'RB', 'G', 'T', 'TE', 'QB', 'C']:
    sub = data[data.market_Pos==pos]
    px = 'wAVpG'
    py = 'max_Cap'# 'next_Cap'
    
    # gather fit data
    cutoff_percentile = pos_cutoffs(pos, py.split('_')[0])
    cutoff_value = np.percentile(sub[sub[py]>0][py], [cutoff_percentile])[0]
    fit_data = sub[sub[py]>=cutoff_value].sort_values([px,py])
    
    # prepare for model
    mx = sm.add_constant(fit_data[px])
    sq_resp = np.sum(fit_data[py]**2) # normalize error

    # statsmodels RLM
    mx = sm.add_constant(fit_data[px])
    sq_resp = np.sum(fit_data[py]**2)
    mod = sm.RLM(fit_data[py], mx).fit()
    pred = mod.predict(mx)
    
    # basic info
    fit_summary.loc[pos,'total'] = sub.shape[0]
    fit_summary.loc[pos,'percentile_cutoff'] = cutoff_percentile
    fit_summary.loc[pos,'cap_cutoff'] = round(cutoff_value,2)
    fit_summary.loc[pos,'market_total'] = fit_data.shape[0]
    fit_summary.loc[pos,'fit_slope'] = mod.params[px]
    fit_summary.loc[pos,'fit_intercept'] = mod.params['const']
    fit_summary.loc[pos,'fit_wAVpG_min'] = round(fit_data[px].min(),2)
    fit_summary.loc[pos,'fit_wAVpG_max'] = round(fit_data[px].max(),2)
    fit_summary.loc[pos,f'fit_{py}_min'] = round(fit_data[py].min(),2)
    fit_summary.loc[pos,f'fit_{py}_max'] = round(fit_data[py].max(),2) 
    fit_summary.loc[pos,'fit_relerror'] = round(100*np.sum(mod.resid**2)/sq_resp,2)
    
    # fun adds
    waste = fit_data.loc[(fit_data[py]-pred).sort_values(ascending=False).index[0]]
    waste = market_data[(market_data.Player==waste.Player)&(market_data.Perc_Cap==waste[py])]
    value = fit_data.loc[(fit_data[py]-pred).sort_values(ascending=True).index[0]]
    value = market_data[(market_data.Player==value.Player)&(market_data.Perc_Cap==value[py])]
    fit_summary.loc[pos,'value_contract'] = f"{value.Player.values[0]}, {value.Perc_Cap.values[0]}% ({value.Year.values[0]})"
    fit_summary.loc[pos,'luxury_contract'] = f"{waste.Player.values[0]}, {waste.Perc_Cap.values[0]}% ({waste.Year.values[0]})"

In [ ]:
fit_summary['total'] = fit_summary['total'].astype(int)
fit_summary['percentile_cutoff'] = fit_summary['percentile_cutoff'].astype(int)

In [ ]:
fit_summary.to_csv(Path('market based position groups', 'tables','max-contract_market_fits.csv'))

### Graphs

#### Market Fit
 - once for second contract `py='next_Cap'`, one for max contract `py='max_Cap'`

In [ ]:
for pos in ['WR', 'CB', 'iDL', 'LB', 'EDGE', 'S', 'RB', 'G', 'T', 'TE', 'QB', 'C']:
    sub = data[data.market_Pos==pos]

    px = 'wAVpG'
    py = 'next_Cap'
    plt.figure(figsize=(8,6), tight_layout=True, dpi=100)
    sns.scatterplot(sub, x = px, y=py, size=py,
                    hue=py, palette='turbo')

    # curve fits, position specific percentile cutoffs
    cutoff_percentile = pos_cutoffs(pos, py.split('_')[0])      
    cutoff_value = np.percentile(sub[sub[py]>0][py], [cutoff_percentile])[0]
    fit_data = sub[sub[py]>=cutoff_value].sort_values([px,py])
    xb = plt.xlim()
    x = np.linspace(fit_data[px].min(),fit_data[px].max(),50)
    sq_resp = np.sum(fit_data[py]**2) # normalize error

    # statsmodels NegBinom, Gamma
    mx = sm.add_constant(fit_data[px])
    fx3 = sm.RLM(fit_data[py], mx).fit()
    plt.plot(x, fx3.predict(sm.add_constant(x)),'--', linewidth=2, color='darkorchid')
    plt.text(xb[1]-.01, cutoff_value+1, f'RLM {round(100*np.sum(fx3.resid**2)/sq_resp,2)}',  
             color='darkorchid', ha='right')

    # polynomial
    poly_fit_3 = np.polyfit(fit_data[px], fit_data[py], 3,  full=True) 
    plt.plot(x, np.poly1d(poly_fit_3[0])(x), '--', linewidth=2, color='deeppink')
    plt.text(xb[1]-.01, cutoff_value+0.5, f'3d poly {round(100*poly_fit_3[1][0]/sq_resp,2)}',  
             color='deeppink', ha='right')

    # smoothed spline
    smooth_factor = 5 if pos == 'QB' else 2
    tck_s = splrep(fit_data[px], fit_data[py], s=fit_data.shape[0]*smooth_factor,)
    # plt.plot(x, BSpline(*tck)(x), 'g-', label='BS 0')
    plt.plot(x, BSpline(*tck_s)(x), '--', color='k')
    plt.text(xb[1]-.01, cutoff_value+0.1, f'Splines {round(100*np.sum((BSpline(*tck_s)(fit_data[px]) - fit_data[py])**2)/sq_resp,2)}',  
             color='k', ha='right')


    plt.text(xb[0]+.01, cutoff_value, f'{cutoff_percentile}th percentile\n{cutoff_value:.1f}% cap',
             va='center', fontsize='small', color='grey')

    plt.plot(xb, [cutoff_value]*2, 'k:', linewidth=0.5)
    plt.xlim(xb)

    plt.title(f'{pos}s drafted since 2011\nwith at least one veteran contract')
    plt.xlabel('weighted AV per Game')
    plt.ylabel('Second Contract %Cap')

    plt.savefig(Path('market based position groups', 'graphs',
                     'Second Contract Fits', f'{pos}_market-fit-nextCap.png'), 
                bbox_inches='tight', dpi=200)
    plt.show()

#### Market Explore

In [ ]:
for pos in ['WR', 'CB', 'iDL', 'LB', 'EDGE', 'S', 'RB', 'G', 'T', 'TE', 'QB', 'C']:
    sub = data[data.market_Pos==pos]
    
    px = 'wAVpG'
    py = 'max_Cap' # next_Cap, delta
    plt.figure(figsize=(8,6), dpi=100)
    sns.scatterplot(sub, x = px, y=py, size=py, hue=py, 
                    palette='turbo')
    plt.xlabel('weighted AV per Game')
    plt.title(f'{pos}s drafted since 2011\nwith at least one veteran contract')
    xb = plt.xlim()

    labels = []
    for label_col in [px,py,'max_Cap']:
        for ind in sub.sort_values(label_col, ascending=False).head().index:
            labels.append(ind)

    cutoff_percentile = 85 if sub.shape[0] > 120 else 75
    cutoff_value = np.percentile(sub[sub[py]>0][py], [cutoff_percentile])[0]
    labels.extend(sub[sub.delta<=cutoff_value].sort_values('wAVpG', ascending=False).index[:3])
    labels.extend(sub[sub.delta>=cutoff_value].sort_values('wAVpG', ascending=True).index[:3])

    labels = set(labels)
    for ind in labels:
        plt.text(sub.loc[ind,px], sub.loc[ind,py], sub.loc[ind,'Player'], 
                 fontsize='6', fontweight='bold', ha='center') 

    # plt.ylabel('Second Contract - Rookie Deal')
    # plt.ylabel('Second Contract %Cap')
    plt.ylabel('highest vet contract\n% cap')
    
    plt.legend(title='% Cap')
    plt.xlim(xb)
    plt.show()
    plt.savefig(Path('market based position groups', 'graphs',
                     'Market Explore', f'{pos}_max-cap.png'), 
                bbox_inches='tight')